In [25]:
def registry(registry_key):
    """Return a metaclass that enables fetching subclasses by an arbitrary key.

    Args:
        registry_key (str): class attribute to use for indexing/retrieving subclasses
    Returns:
        type: metaclass to be used in the base class

    Example usage:
        >>> class Base(metaclass=registry('platform')):
        ...    platform = None
        >>>
        >>> class Youtube(Base):
        ...    platform = 'yt'
        >>>
        >>> class Facebook(Base):
        ...   platform = 'fb'
        >>>

        >>> Base.registry['yt']
        <class 'swissarmy.class_registry.Youtube'>

        >>> Base.registry['fb']
        <class 'swissarmy.class_registry.Facebook'>
    """

    class Registry(type):
        def __init__(cls, name, bases, classdict):
            super(Registry, cls).__init__(name, bases, classdict)
            if not hasattr(cls, 'registry'):
                cls.registry = {}
            key = classdict.get(registry_key, None)
            if key is not None:
                if key in cls.registry:
                    raise Exception('Duplicate registry key found: %s' % key)
                cls.registry[key] = cls

    return Registry


In [33]:
class Op(metaclass=registry('opcode')):
    opcode = None

    @classmethod
    def run(cls, mem, ip, inp):
        cls.modes = '0000' + str(mem[ip] // 100)
        cls.ip = ip
        cls.mem = mem
        cls.inp = inp
        new_ip = cls.func()
        return new_ip

    @classmethod
    def val(cls, arg_num):
        val = cls.param(arg_num)
        if cls.modes[-arg_num] == '0':
            return cls.mem[val]
        else:
            return val

    @classmethod
    def param(cls, arg_num):
        return cls.mem[cls.ip + arg_num]
    
    @classmethod
    def store(cls, value, arg_num=3):
        cls.mem[cls.param(arg_num)] = value
        
    @classmethod
    def func(cls):
        pass

class Add(Op):
    opcode = 1
    
    @classmethod
    def func(cls):
        cls.store(cls.val(1) + cls.val(2))
        return cls.ip + 4

class Mult(Op):
    opcode = 2
    
    @classmethod
    def func(cls):
        cls.store(cls.val(1) * cls.val(2))
        return cls.ip + 4

class Halt(Op):
    opcode = 99

    @classmethod
    def func(cls):
        return None

class Input(Op):
    opcode = 3
    
    @classmethod
    def func(cls):
        cls.store(cls.inp, 1)
        return cls.ip + 2

class Output(Op):
    opcode = 4
    
    @classmethod
    def func(cls):
        print(cls.val(1))
        return cls.ip + 2

class JumpIfTrue(Op):
    opcode = 5
    
    @classmethod
    def func(cls):
        if cls.val(1):
            return cls.val(2)
        return cls.ip + 3

class JumpIfFalse(Op):
    opcode = 6
    
    @classmethod
    def func(cls):
        if cls.val(1):
            return cls.ip + 3
        return cls.val(2)

class LessThan(Op):
    opcode = 7
    
    @classmethod
    def func(cls):
        if cls.val(1) < cls.val(2):
            cls.store(1)
        else:
            cls.store(0)
        return cls.ip + 4

class Equals(Op):
    opcode = 8
    
    @classmethod
    def func(cls):
        if cls.val(1) == cls.val(2):
            cls.store(1)
        else:
            cls.store(0)
        return cls.ip + 4
            
class Test:
    def __init__(self, program, inp):
        self.program = program
        self.inp = inp
    
    def run(self):
        mem = [int(o) for o in self.program.strip().split(',')]
        ip = 0
        while ip is not None:
            # parse op
            opcode = mem[ip] % 100
            op = Op.registry[opcode]
            ip = op.run(mem, ip, self.inp)
        print('Done!')
            

In [31]:
with open('input.txt', 'r') as f:
    program = f.read()
Test(program, 1).run()

0
0
0
0
0
0
0
0
0
15259545
Done!


In [32]:
Test(program, 5).run()

7616021
Done!
